In [20]:
!apt-get update -y
!apt-get install openjdk-11-jdk -y
!java -version

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [59.5 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,107 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,446 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,163 kB]
Get:13 http://archive.ubun

In [21]:
!pip install apache-flink


In [24]:
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.common import Types
from pyflink.datastream.functions import MapFunction
import requests
import json
import time
from datetime import datetime

In [25]:
GITHUB_API_URL = "https://api.github.com/search/repositories?q=is:public&sort=issues&order=desc&per_page=50"
QUERY_PARAMS = {
    "q": "is:public",  # filter for public repositories
    "sort": "stars",    # sort by stars or issues
    "order": "desc",    # descending order (most stars or issues first)
    "per_page": 50,     # limit to 50 repositories per call
}

def fetch_github_repositories():
    # Make the API call to fetch repositories data
    response = requests.get(GITHUB_API_URL, params=QUERY_PARAMS)
    if response.status_code == 200:
        return response.json()['items']  # Returns the list of repositories
    else:
        print(f"Error fetching data from GitHub API: {response.status_code}")
        return []

In [26]:
class GithubRepositoryMapFunction(MapFunction):
    def map(self, value):
        # We will process each repository's information here
        repo_data = {
            "id": value['id'],
            "name": value['name'],
            "full_name": value['full_name'],
            "language": value['language'],
            "stargazers_count": value['stargazers_count'],
            "forks_count": value['forks_count'],
            "open_issues_count": value['open_issues_count'],
            "created_at": value['created_at'],
            "updated_at": value['updated_at'],
            "pushed_at": value['pushed_at'],
        }
        return repo_data


In [27]:
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.connectors import FlinkKafkaConsumer
from pyflink.common.serialization import SimpleStringSchema
from pyflink.common.typeinfo import Types
from pyflink.common import Row
import json

def process_event(event):
    # Example of aprocessing: Parse the event and filter or transform it
    try:
        data = json.loads(event)  # Convert JSON string to Python dict
        if 'push' in data['event']:  # Example: Filter push events
            return Row(data['actor']['login'], data['repo']['name'], data['event'])
    except Exception as e:
        print(f"Error processing event: {e}")
        return None

def main():
    # Set up the Flink streaming environment
    env = StreamExecutionEnvironment.get_execution_environment()

    # Kafka consumer properties
    properties = {
        'bootstrap.servers': 'localhost:9092',  # Kafka broker address
        'group.id': 'github-events-group',
    }

    # Create a Flink Kafka Consumer
    kafka_source = FlinkKafkaConsumer(
        topics='github-events',  # Kafka topic
        deserialization_schema=SimpleStringSchema(),  # Deserialize as plain text
        properties=properties
    )

    # Add the Kafka source to the environment
    stream = env.add_source(kafka_source)

    # Process each event (parse JSON, filter events, etc.)
    processed_stream = stream.map(process_event, output_type=Types.ROW([Types.STRING(), Types.STRING(), Types.STRING()]))

    # Print processed stream (or store to a database, etc.)
    processed_stream.print()

    # Execute the Flink job
    env.execute('GitHub Event Stream Processing')


In [28]:
if __name__ == '__main__':
    main()

TypeError: Could not found the Java class 'org.apache.flink.streaming.connectors.kafka.FlinkKafkaConsumer'. The Java dependencies could be specified via command line argument '--jarfile' or the config option 'pipeline.jars'